In [1]:
import numpy as np

In [2]:
arr = np.zeros((3, 4, 5, 6, 7, 8))
new_arr = arr.reshape(*arr.shape[:2], -1, *arr.shape[-2:])
new_arr.shape

(3, 4, 30, 7, 8)

In [3]:
x = np.random.rand(2, 3)
x

array([[0.26887294, 0.55044774, 0.21909798],
       [0.54125327, 0.40864761, 0.97505898]])

In [4]:
k_size = 3
chan_in = 1
chan_out = 2
stride = 1

batch = 4
length = 8

In [5]:
params = np.random.rand(k_size, chan_in, chan_out)
X = np.random.rand(batch, length, chan_in)

In [6]:

d_out = (length - k_size) // stride + 1
d_out, params.shape, X.shape

(6, (3, 1, 2), (4, 8, 1))

In [7]:
out = np.zeros((batch, d_out, chan_out))

In [8]:
for img in range(batch):
    for cout in range(chan_out):
        for i in range(d_out):
            for j in range(k_size):
                for c in range(chan_in):
                    out[img, i, cout] += params[j, c, cout] * X[img, i*stride+j, c]
out

array([[[0.31976792, 0.72313456],
        [0.14339892, 0.41898735],
        [0.33507445, 0.58459576],
        [0.3041578 , 0.77936684],
        [0.2878784 , 0.61488138],
        [0.42912371, 0.88744567]],

       [[0.59292037, 1.3616896 ],
        [0.55844652, 1.26657619],
        [0.27416838, 0.67549366],
        [0.508804  , 0.93293328],
        [0.61581786, 1.44947122],
        [0.49778097, 1.16409925]],

       [[0.47918488, 0.81095942],
        [0.25399495, 0.79034986],
        [0.12611547, 0.20980697],
        [0.20703997, 0.38733281],
        [0.21387251, 0.52785864],
        [0.28636341, 0.59866675]],

       [[0.4807901 , 0.79675979],
        [0.28580554, 0.84272475],
        [0.30643811, 0.57408696],
        [0.52453278, 1.09764516],
        [0.52774841, 1.2623004 ],
        [0.23081767, 0.58684299]]])

In [10]:
for i in range(d_out):
    out[:, i, :] = np.sum(
        X[:, i * stride : k_size + i * stride + 1, :, np.newaxis]
        * params[np.newaxis, :, :, :],
        axis=(1, 2),
    )
out


array([[[0.31976792, 0.72313456],
        [0.14339892, 0.41898735],
        [0.33507445, 0.58459576],
        [0.3041578 , 0.77936684],
        [0.2878784 , 0.61488138],
        [0.42912371, 0.88744567]],

       [[0.59292037, 1.3616896 ],
        [0.55844652, 1.26657619],
        [0.27416838, 0.67549366],
        [0.508804  , 0.93293328],
        [0.61581786, 1.44947122],
        [0.49778097, 1.16409925]],

       [[0.47918488, 0.81095942],
        [0.25399495, 0.79034986],
        [0.12611547, 0.20980697],
        [0.20703997, 0.38733281],
        [0.21387251, 0.52785864],
        [0.28636341, 0.59866675]],

       [[0.4807901 , 0.79675979],
        [0.28580554, 0.84272475],
        [0.30643811, 0.57408696],
        [0.52453278, 1.09764516],
        [0.52774841, 1.2623004 ],
        [0.23081767, 0.58684299]]])